# Descripción General del Proyecto:
    
    - Obtener los datos estadísticos de los jugadores de la NBA del 1980 a 2021
    - Análisis exploratorio para identificar comportamiento de algunas variables y su relación con el premio MVP
    - Entrenar un modelo que permita pronosticar el MVP del 2021

# Obtención de Datos

In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
driver = webdriver.Chrome()

In [3]:
years = list(range(2010,2015,1))
stats = []
MVP = []

for year in years:
    url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html)
    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]
    MVP_W = soup.findAll('p')[3].find('a').text
    MVP.append([MVP_W, year])
    #player_stats[i].append(year)
    stats.append(player_stats)

In [4]:
seasons = list(range(len(stats)))

for season, year in enumerate(years):
        for players in list(range(len(stats[season]))):
            stats[season][players].append(year)

In [5]:
data = pd.DataFrame.from_records(stats)

In [6]:
data = pd.DataFrame()

for i in range(len(stats)):
    for l in range(len(stats[i])):
        d = pd.DataFrame(stats[i][l]).transpose()
        data = data.append(d, ignore_index=True)

In [7]:
driver = webdriver.Chrome()
url = "https://www.basketball-reference.com/leagues/NBA_2020_per_game.html"
driver.get(url)
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html)
headers = [th.getText() for th in soup.findAll('thead')[0].findAll('tr')[0].findAll('th')]
headers = headers[1:]
headers.append('year')

In [10]:
advanced = []

for year in years:
    url = "https://www.basketball-reference.com/leagues/NBA_{}_advanced.html".format(year)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html)
    rows = soup.findAll('tr')[1:]
    player_advanced = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]
    advanced.append(player_advanced)

In [11]:
seasons = list(range(len(advanced)))

for season, year in enumerate(years):
        for players in list(range(len(advanced[season]))):
            advanced[season][players].append(year)

In [12]:
data_advanced = pd.DataFrame.from_records(advanced)

In [13]:
data_advanced = pd.DataFrame()

for i in range(len(advanced)):
    for l in range(len(advanced[i])):
        d = pd.DataFrame(advanced[i][l]).transpose()
        data_advanced = data_advanced.append(d, ignore_index=True)

In [14]:
driver = webdriver.Chrome()
url = "https://www.basketball-reference.com/leagues/NBA_2020_advanced.html"
driver.get(url)
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html)
headers_advanced = [th.getText() for th in soup.findAll('thead')[0].findAll('tr')[0].findAll('th')]
headers_advanced = headers_advanced[1:]
headers_advanced.append('year')

In [15]:
data.columns = headers
data_advanced.columns = headers_advanced

In [16]:
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,year
0,Boston Celtics,(46-28),F,$,2010,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Toronto Raptors,(40-32),F,$,2010,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brooklyn Nets,(38-35),F,$,2010,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,New York Knicks,(31-42),F,$,2010,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Milwaukee Bucks,(45-27),F,$,2010,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
data_advanced.head()

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,year
0,Boston Celtics,(46-28),F,$,2010,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Toronto Raptors,(40-32),F,$,2010,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brooklyn Nets,(38-35),F,$,2010,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,New York Knicks,(31-42),F,$,2010,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Milwaukee Bucks,(45-27),F,$,2010,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
MVP_DF = pd.DataFrame(MVP, columns=['jugador','año'])

# Ingeniería de Variables

In [19]:
data = data.dropna()
data_advanced = data_advanced.dropna()

In [20]:
data.reset_index(drop=True,inplace=True)

In [21]:
data_advanced.reset_index(drop=True,inplace=True)

In [22]:
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,year
0,Arron Afflalo,SG,24,DEN,82,75,27.1,3.3,7.1,.465,...,0.7,2.4,3.1,1.7,0.6,0.4,0.9,2.7,8.8,2010
1,Alexis Ajinça,C,21,CHA,6,0,5.0,0.8,1.7,.500,...,0.2,0.5,0.7,0.0,0.2,0.2,0.3,0.8,1.7,2010
2,LaMarcus Aldridge,PF,24,POR,78,78,37.5,7.4,15.0,.495,...,2.5,5.6,8.0,2.1,0.9,0.6,1.3,3.0,17.9,2010
3,Joe Alexander,SF,23,CHI,8,0,3.6,0.1,0.8,.167,...,0.3,0.4,0.6,0.3,0.1,0.1,0.0,1.1,0.5,2010
4,Malik Allen,PF,31,DEN,51,3,8.9,0.9,2.3,.397,...,0.7,0.9,1.6,0.3,0.2,0.1,0.4,1.3,2.1,2010


In [23]:
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,year
0,Arron Afflalo,SG,24,DEN,82,75,27.1,3.3,7.1,.465,...,0.7,2.4,3.1,1.7,0.6,0.4,0.9,2.7,8.8,2010
1,Alexis Ajinça,C,21,CHA,6,0,5.0,0.8,1.7,.500,...,0.2,0.5,0.7,0.0,0.2,0.2,0.3,0.8,1.7,2010
2,LaMarcus Aldridge,PF,24,POR,78,78,37.5,7.4,15.0,.495,...,2.5,5.6,8.0,2.1,0.9,0.6,1.3,3.0,17.9,2010
3,Joe Alexander,SF,23,CHI,8,0,3.6,0.1,0.8,.167,...,0.3,0.4,0.6,0.3,0.1,0.1,0.0,1.1,0.5,2010
4,Malik Allen,PF,31,DEN,51,3,8.9,0.9,2.3,.397,...,0.7,0.9,1.6,0.3,0.2,0.1,0.4,1.3,2.1,2010


In [24]:
data_advanced.head()

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,year
0,Arron Afflalo,SG,24,DEN,82,2221,10.9,.576,.426,.168,...,2.8,1.4,4.3,.092,,-0.2,-0.2,-0.4,0.9,2010
1,Alexis Ajinça,C,21,CHA,6,30,6.3,.479,.000,.100,...,-0.1,0.0,0.0,-0.013,,-6.3,1.0,-5.3,0.0,2010
2,LaMarcus Aldridge,PF,24,POR,78,2922,18.2,.535,.014,.260,...,5.5,3.3,8.8,.145,,1.4,-0.2,1.2,2.3,2010
3,Joe Alexander,SF,23,CHI,8,29,2.8,.273,.167,.500,...,0.0,0.0,0.0,.030,,-9.1,0.9,-8.3,0.0,2010
4,Malik Allen,PF,31,DEN,51,456,5.9,.431,.052,.112,...,-0.3,0.3,0.1,.009,,-4.7,-1.0,-5.7,-0.4,2010


In [26]:
data['Player'] = data['Player'].str.replace('*','')
data_advanced['Player'] = data_advanced['Player'].str.replace('*','')

In [27]:
data_advanced.rename({'Pos':'Pos_a',
                      'Age':'Age_a',
                      'Tm':'Tm_a',
                      'G':'G_a',
                      'MP':'MP_a',
                      'year':'year_a',
                      'Player':'Player_a'},
                     axis=1,
                     inplace=True)

In [28]:
data_advanced.head()

,Player_a,Pos_a,Age_a,Tm_a,G_a,MP_a,PER,TS%,3PAr,FTr,...,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,year_a
0,Arron Afflalo,SG,24,DEN,82,2221,10.9,.576,.426,.168,...,2.8,1.4,4.3,.092,,-0.2,-0.2,-0.4,0.9,2010
1,Alexis Ajinça,C,21,CHA,6,30,6.3,.479,.000,.100,...,-0.1,0.0,0.0,-0.013,,-6.3,1.0,-5.3,0.0,2010
2,LaMarcus Aldridge,PF,24,POR,78,2922,18.2,.535,.014,.260,...,5.5,3.3,8.8,.145,,1.4,-0.2,1.2,2.3,2010
3,Joe Alexander,SF,23,CHI,8,29,2.8,.273,.167,.500,...,0.0,0.0,0.0,.030,,-9.1,0.9,-8.3,0.0,2010
4,Malik Allen,PF,31,DEN,51,456,5.9,.431,.052,.112,...,-0.3,0.3,0.1,.009,,-4.7,-1.0,-5.7,-0.4,2010


In [29]:
nba_complete = pd.merge(data,
                        data_advanced,
                        how='left', 
                        left_on=['Player','Pos','Tm','G','Age','year'], 
                        right_on=['Player_a','Pos_a','Tm_a','G_a','Age_a','year_a'])

In [30]:
data

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,year
0,Arron Afflalo,SG,24,DEN,82,75,27.1,3.3,7.1,.465,...,0.7,2.4,3.1,1.7,0.6,0.4,0.9,2.7,8.8,2010
1,Alexis Ajinça,C,21,CHA,6,0,5.0,0.8,1.7,.500,...,0.2,0.5,0.7,0.0,0.2,0.2,0.3,0.8,1.7,2010
2,LaMarcus Aldridge,PF,24,POR,78,78,37.5,7.4,15.0,.495,...,2.5,5.6,8.0,2.1,0.9,0.6,1.3,3.0,17.9,2010
3,Joe Alexander,SF,23,CHI,8,0,3.6,0.1,0.8,.167,...,0.3,0.4,0.6,0.3,0.1,0.1,0.0,1.1,0.5,2010
4,Malik Allen,PF,31,DEN,51,3,8.9,0.9,2.3,.397,...,0.7,0.9,1.6,0.3,0.2,0.1,0.4,1.3,2.1,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,Tony Wroten,PG,20,PHI,72,16,24.5,4.8,11.2,.427,...,1.0,2.2,3.2,3.0,1.1,0.2,2.8,2.1,13.0,2014
2934,Nick Young,SG,28,LAL,64,9,28.3,6.0,13.9,.435,...,0.5,2.1,2.6,1.5,0.7,0.2,1.5,2.4,17.9,2014
2935,Thaddeus Young,PF,25,PHI,79,78,34.4,7.4,16.2,.454,...,2.1,3.9,6.0,2.3,2.1,0.5,2.1,2.7,17.9,2014
2936,Cody Zeller,C,21,CHA,82,3,17.3,2.1,4.9,.426,...,1.4,2.9,4.3,1.1,0.5,0.5,1.1,2.1,6.0,2014


In [31]:
nba_complete.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,year_a
0,Arron Afflalo,SG,24,DEN,82,75,27.1,3.3,7.1,.465,...,2.8,1.4,4.3,.092,,-0.2,-0.2,-0.4,0.9,2010
1,Alexis Ajinça,C,21,CHA,6,0,5.0,0.8,1.7,.500,...,-0.1,0.0,0.0,-0.013,,-6.3,1.0,-5.3,0.0,2010
2,LaMarcus Aldridge,PF,24,POR,78,78,37.5,7.4,15.0,.495,...,5.5,3.3,8.8,.145,,1.4,-0.2,1.2,2.3,2010
3,Joe Alexander,SF,23,CHI,8,0,3.6,0.1,0.8,.167,...,0.0,0.0,0.0,.030,,-9.1,0.9,-8.3,0.0,2010
4,Malik Allen,PF,31,DEN,51,3,8.9,0.9,2.3,.397,...,-0.3,0.3,0.1,.009,,-4.7,-1.0,-5.7,-0.4,2010


In [32]:
nba_complete.tail()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,year_a
2933,Tony Wroten,PG,20,PHI,72,16,24.5,4.8,11.2,.427,...,-1.8,0.9,-0.9,-0.025,,-1.9,-1.8,-3.7,-0.7,2014
2934,Nick Young,SG,28,LAL,64,9,28.3,6.0,13.9,.435,...,2.3,0.3,2.5,.067,,1.4,-1.9,-0.5,0.7,2014
2935,Thaddeus Young,PF,25,PHI,79,78,34.4,7.4,16.2,.454,...,1.2,2.3,3.5,.062,,0.4,-0.5,-0.1,1.3,2014
2936,Cody Zeller,C,21,CHA,82,3,17.3,2.1,4.9,.426,...,0.4,2.2,2.6,.090,,-2.9,0.9,-2.0,0.0,2014
2937,Tyler Zeller,C,24,CLE,70,9,15.0,2.2,4.1,.538,...,1.5,1.1,2.6,.120,,-1.8,-0.4,-2.2,-0.1,2014


In [33]:
nba_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2938 entries, 0 to 2937
Data columns (total 59 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Player    2938 non-null   object
 1   Pos       2938 non-null   object
 2   Age       2938 non-null   object
 3   Tm        2938 non-null   object
 4   G         2938 non-null   object
 5   GS        2938 non-null   object
 6   MP        2938 non-null   object
 7   FG        2938 non-null   object
 8   FGA       2938 non-null   object
 9   FG%       2938 non-null   object
 10  3P        2938 non-null   object
 11  3PA       2938 non-null   object
 12  3P%       2938 non-null   object
 13  2P        2938 non-null   object
 14  2PA       2938 non-null   object
 15  2P%       2938 non-null   object
 16  eFG%      2938 non-null   object
 17  FT        2938 non-null   object
 18  FTA       2938 non-null   object
 19  FT%       2938 non-null   object
 20  ORB       2938 non-null   object
 21  DRB       2938

In [34]:
nba_complete.columns[4:]

Index(['G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
       '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'year', 'Player_a', 'Pos_a', 'Age_a', 'Tm_a',
       'G_a', 'MP_a', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%',
       'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', ' ', 'OWS', 'DWS', 'WS',
       'WS/48', ' ', 'OBPM', 'DBPM', 'BPM', 'VORP', 'year_a'],
      dtype='object')

In [43]:
nba_complete_features = nba_complete[['Player',
                                      'Pos',
                                      'Tm_a',
                                      'Age',
                                      'G',
                                      'year',
                                      'PTS',
                                      'AST',
                                      'STL',
                                      'TRB',
                                      'BLK',
                                      'TOV',
                                      'WS',
                                      'PER',
                                      'VORP',
                                      'BPM',
                                      'WS/48',
                                      'FG',
                                      'FG%',
                                      'FGA',
                                      '3P',
                                      '3PA',
                                      '3P%',
                                      '2P',
                                      '2PA',
                                      '2P%',
                                      'FT',
                                      'FTA']]

In [44]:
nba_complete_features.columns[2:]

Index(['Tm_a', 'Age', 'G', 'year', 'PTS', 'AST', 'STL', 'TRB', 'BLK', 'TOV',
       'WS', 'PER', 'VORP', 'BPM', 'WS/48', 'FG', 'FG%', 'FGA', '3P', '3PA',
       '3P%', '2P', '2PA', '2P%', 'FT', 'FTA'],
      dtype='object')

In [45]:
nba_complete_features.head()

,Player,Pos,Tm_a,Age,G,year,PTS,AST,STL,TRB,...,FG%,FGA,3P,3PA,3P%,2P,2PA,2P%,FT,FTA
0,Arron Afflalo,SG,DEN,24,82,2010,8.8,1.7,0.6,3.1,...,.465,7.1,1.3,3.0,.434,2.0,4.1,.488,0.9,1.2
1,Alexis Ajinça,C,CHA,21,6,2010,1.7,0.0,0.2,0.7,...,.500,1.7,0.0,0.0,,0.8,1.7,.500,0.0,0.2
2,LaMarcus Aldridge,PF,POR,24,78,2010,17.9,2.1,0.9,8.0,...,.495,15.0,0.1,0.2,.313,7.4,14.8,.498,2.9,3.9
3,Joe Alexander,SF,CHI,23,8,2010,0.5,0.3,0.1,0.6,...,.167,0.8,0.0,0.1,.000,0.1,0.6,.200,0.3,0.4
4,Malik Allen,PF,DEN,31,51,2010,2.1,0.3,0.2,1.6,...,.397,2.3,0.0,0.1,.167,0.9,2.2,.409,0.2,0.3


In [46]:
columnas = nba_complete_features.columns[3:]

for st in columnas:
    nba_complete_features[st] = pd.to_numeric(nba_complete_features[st], errors='coerce')
    

In [47]:
nba_complete_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2938 entries, 0 to 2937
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  2938 non-null   object 
 1   Pos     2938 non-null   object 
 2   Tm_a    2938 non-null   object 
 3   Age     2938 non-null   int64  
 4   G       2938 non-null   int64  
 5   year    2938 non-null   int64  
 6   PTS     2938 non-null   float64
 7   AST     2938 non-null   float64
 8   STL     2938 non-null   float64
 9   TRB     2938 non-null   float64
 10  BLK     2938 non-null   float64
 11  TOV     2938 non-null   float64
 12  WS      2938 non-null   float64
 13  PER     2936 non-null   float64
 14  VORP    2938 non-null   float64
 15  BPM     2938 non-null   float64
 16  WS/48   2936 non-null   float64
 17  FG      2938 non-null   float64
 18  FG%     2921 non-null   float64
 19  FGA     2938 non-null   float64
 20  3P      2938 non-null   float64
 21  3PA     2938 non-null   float64
 22  

In [48]:
nba_complete_features.fillna(0,inplace=True)

In [49]:
nba_complete_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2938 entries, 0 to 2937
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  2938 non-null   object 
 1   Pos     2938 non-null   object 
 2   Tm_a    2938 non-null   object 
 3   Age     2938 non-null   int64  
 4   G       2938 non-null   int64  
 5   year    2938 non-null   int64  
 6   PTS     2938 non-null   float64
 7   AST     2938 non-null   float64
 8   STL     2938 non-null   float64
 9   TRB     2938 non-null   float64
 10  BLK     2938 non-null   float64
 11  TOV     2938 non-null   float64
 12  WS      2938 non-null   float64
 13  PER     2938 non-null   float64
 14  VORP    2938 non-null   float64
 15  BPM     2938 non-null   float64
 16  WS/48   2938 non-null   float64
 17  FG      2938 non-null   float64
 18  FG%     2938 non-null   float64
 19  FGA     2938 non-null   float64
 20  3P      2938 non-null   float64
 21  3PA     2938 non-null   float64
 22  

In [50]:
MVP_ = pd.DataFrame(MVP, columns=['player_mvp', 'year_mvp'])

In [51]:
MVP_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   player_mvp  5 non-null      object
 1   year_mvp    5 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


In [52]:
MVP_ = MVP_[0:41]

In [53]:
MVP_.head()

,player_mvp,year_mvp
0,LeBron James,2010
1,Derrick Rose,2011
2,LeBron James,2012
3,LeBron James,2013
4,Kevin Durant,2014


In [54]:
nba_mvp = pd.merge(nba_complete_features,MVP_, how='left', left_on=['Player','year'], right_on=['player_mvp','year_mvp'])

In [55]:
nba_mvp.fillna('0',inplace=True)

nba_mvp['mvp'] =  np.where(nba_mvp['player_mvp']!='0','1','0')
nba_mvp.head(5)

,Player,Pos,Tm_a,Age,G,year,PTS,AST,STL,TRB,...,3PA,3P%,2P,2PA,2P%,FT,FTA,player_mvp,year_mvp,mvp
0,Arron Afflalo,SG,DEN,24,82,2010,8.8,1.7,0.6,3.1,...,3.0,0.434,2.0,4.1,0.488,0.9,1.2,0,0,0
1,Alexis Ajinça,C,CHA,21,6,2010,1.7,0.0,0.2,0.7,...,0.0,0.000,0.8,1.7,0.500,0.0,0.2,0,0,0
2,LaMarcus Aldridge,PF,POR,24,78,2010,17.9,2.1,0.9,8.0,...,0.2,0.313,7.4,14.8,0.498,2.9,3.9,0,0,0
3,Joe Alexander,SF,CHI,23,8,2010,0.5,0.3,0.1,0.6,...,0.1,0.000,0.1,0.6,0.200,0.3,0.4,0,0,0
4,Malik Allen,PF,DEN,31,51,2010,2.1,0.3,0.2,1.6,...,0.1,0.167,0.9,2.2,0.409,0.2,0.3,0,0,0


In [56]:
nba_mvp.drop(['player_mvp','year_mvp'], axis=1,inplace=True)

In [57]:
nba_mvp[nba_mvp['mvp']=='1']

,Player,Pos,Tm_a,Age,G,year,PTS,AST,STL,TRB,...,FGA,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,mvp
285,LeBron James,SF,CLE,25,76,2010,29.7,8.6,1.6,7.3,...,20.1,1.7,5.1,0.333,8.4,15.0,0.560,7.8,10.2,1
1068,Derrick Rose,PG,CHI,22,81,2011,25.0,7.7,1.0,4.1,...,19.7,1.6,4.8,0.332,7.2,15.0,0.481,5.9,6.9,1
1472,LeBron James,SF,MIA,27,62,2012,27.1,6.2,1.9,7.9,...,18.9,0.9,2.4,0.362,9.1,16.5,0.556,6.2,8.1,1
2009,LeBron James,PF,MIA,28,76,2013,26.8,7.3,1.7,8.0,...,17.8,1.4,3.3,0.406,8.7,14.5,0.602,5.3,7.0,1
2512,Kevin Durant,SF,OKC,25,81,2014,32.0,5.5,1.3,7.4,...,20.8,2.4,6.1,0.391,8.1,14.8,0.549,8.7,9.9,1


In [58]:
nba_mvp['mvp'] = nba_mvp['mvp'].astype(int)

In [59]:
nba_mvp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2938 entries, 0 to 2937
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  2938 non-null   object 
 1   Pos     2938 non-null   object 
 2   Tm_a    2938 non-null   object 
 3   Age     2938 non-null   int64  
 4   G       2938 non-null   int64  
 5   year    2938 non-null   int64  
 6   PTS     2938 non-null   float64
 7   AST     2938 non-null   float64
 8   STL     2938 non-null   float64
 9   TRB     2938 non-null   float64
 10  BLK     2938 non-null   float64
 11  TOV     2938 non-null   float64
 12  WS      2938 non-null   float64
 13  PER     2938 non-null   float64
 14  VORP    2938 non-null   float64
 15  BPM     2938 non-null   float64
 16  WS/48   2938 non-null   float64
 17  FG      2938 non-null   float64
 18  FG%     2938 non-null   float64
 19  FGA     2938 non-null   float64
 20  3P      2938 non-null   float64
 21  3PA     2938 non-null   float64
 22  

In [60]:
nba_mvp.head()

,Player,Pos,Tm_a,Age,G,year,PTS,AST,STL,TRB,...,FGA,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,mvp
0,Arron Afflalo,SG,DEN,24,82,2010,8.8,1.7,0.6,3.1,...,7.1,1.3,3.0,0.434,2.0,4.1,0.488,0.9,1.2,0
1,Alexis Ajinça,C,CHA,21,6,2010,1.7,0.0,0.2,0.7,...,1.7,0.0,0.0,0.000,0.8,1.7,0.500,0.0,0.2,0
2,LaMarcus Aldridge,PF,POR,24,78,2010,17.9,2.1,0.9,8.0,...,15.0,0.1,0.2,0.313,7.4,14.8,0.498,2.9,3.9,0
3,Joe Alexander,SF,CHI,23,8,2010,0.5,0.3,0.1,0.6,...,0.8,0.0,0.1,0.000,0.1,0.6,0.200,0.3,0.4,0
4,Malik Allen,PF,DEN,31,51,2010,2.1,0.3,0.2,1.6,...,2.3,0.0,0.1,0.167,0.9,2.2,0.409,0.2,0.3,0


In [61]:
nba_mvp.rename(columns={'WS/48':'WS48','FG%':'FGperc','3P':'P3','3PA':'P3A','3P%':'P3perc','2P':'P2','2PA':'P2A','2Pperc':'P2%'},inplace=True)

In [62]:
nba_mvp.rename(columns={'2P%':'P2perc'},inplace=True)

In [63]:
nba_mvp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2938 entries, 0 to 2937
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  2938 non-null   object 
 1   Pos     2938 non-null   object 
 2   Tm_a    2938 non-null   object 
 3   Age     2938 non-null   int64  
 4   G       2938 non-null   int64  
 5   year    2938 non-null   int64  
 6   PTS     2938 non-null   float64
 7   AST     2938 non-null   float64
 8   STL     2938 non-null   float64
 9   TRB     2938 non-null   float64
 10  BLK     2938 non-null   float64
 11  TOV     2938 non-null   float64
 12  WS      2938 non-null   float64
 13  PER     2938 non-null   float64
 14  VORP    2938 non-null   float64
 15  BPM     2938 non-null   float64
 16  WS48    2938 non-null   float64
 17  FG      2938 non-null   float64
 18  FGperc  2938 non-null   float64
 19  FGA     2938 non-null   float64
 20  P3      2938 non-null   float64
 21  P3A     2938 non-null   float64
 22  

In [64]:
nba_mvp.head()

,Player,Pos,Tm_a,Age,G,year,PTS,AST,STL,TRB,...,FGA,P3,P3A,P3perc,P2,P2A,P2perc,FT,FTA,mvp
0,Arron Afflalo,SG,DEN,24,82,2010,8.8,1.7,0.6,3.1,...,7.1,1.3,3.0,0.434,2.0,4.1,0.488,0.9,1.2,0
1,Alexis Ajinça,C,CHA,21,6,2010,1.7,0.0,0.2,0.7,...,1.7,0.0,0.0,0.000,0.8,1.7,0.500,0.0,0.2,0
2,LaMarcus Aldridge,PF,POR,24,78,2010,17.9,2.1,0.9,8.0,...,15.0,0.1,0.2,0.313,7.4,14.8,0.498,2.9,3.9,0
3,Joe Alexander,SF,CHI,23,8,2010,0.5,0.3,0.1,0.6,...,0.8,0.0,0.1,0.000,0.1,0.6,0.200,0.3,0.4,0
4,Malik Allen,PF,DEN,31,51,2010,2.1,0.3,0.2,1.6,...,2.3,0.0,0.1,0.167,0.9,2.2,0.409,0.2,0.3,0


In [65]:
driver.close()

In [105]:
import pyodbc
import sqlalchemy

In [113]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-HAPH02BA;'
                      'Database=master;'
                      'Trusted_Connection=yes;')

In [114]:
cursor = conn.cursor()

In [116]:
engine = sqlalchemy.create_engine('mssql+pyodbc://' + 'LAPTOP-HAPH02BA' + '/' +'master'+ "?" + 'driver=SQL Server Native Client 11.0')

In [ ]:
nba_mvp.to_sql(name = 'NBA',con = engine,
          if_exists = 'append',schema = 'master',
          index = False)